In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
df = pd.read_csv("dog_data_09032022.csv")
df.tail(3)

## Data Cleaning

In [ ]:
# Let's tackle one column after another
# drop first unnamed column
df = df.drop(df.columns[0], axis=1)
df.tail(2)

In [ ]:
# clean height
concat_height = ""
for h in df.height:
    concat_height = concat_height + str(h)
result = ''.join([i for i in concat_height if not i.isdigit()])

# they either have
# 1) one unit
# 2) male & female
# 3) Toy & Miniature & Standard


In [ ]:
concat_weight = ""
for h in df.life:
    concat_weight = concat_weight + str(h)

result = ''.join([i for i in concat_weight if not i.isdigit()])

# they either have
# 1) one unit
# 2) male & female
# 3) Toy & Miniature & Standard


In [ ]:
df['temp'] = df.height.apply(lambda x: len(str(x).split('~')))

In [ ]:
df[df['temp'] == 5]

In [ ]:
# I can either
# if one unit -> male -> standard
sns.heatmap(df.corr())
df.corr()